# Utilisation du modèle

Importation des modules

In [ ]:
import pickle

import cv2
import mediapipe as mp
import numpy as np

Chargement de la configuration

In [ ]:
import json

with open('config.json', 'r') as f:
   config = json.load(f)

MODEL_FILE = config['model']['file']
DATASET_DEFINITION = config['dataset']['definition']
DATASET_IMAGES = config['dataset']['images']
DATASET_SHOWIMAGES = config['dataset']['show_images']
UI_RECTANGLE = config['ui']['rectangle']
UI_TEXT = config['ui']['text']

Chargement du modèle permettant de détecter les mains pour afficher les points sur les mains

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.6)

Chargement du modèle

In [ ]:
model_dict = pickle.load(open(MODEL_FILE, 'rb'))
model = model_dict['model']

Fonction permettant de récupérer les positions X et Y de chaque point de la main.

In [ ]:
def get_hand_data(multi_hand_landmarks):
    data = []
    for landmark in multi_hand_landmarks:
        data.append(landmark.x)
        data.append(landmark.y)
    return data

Fonction permettant d'afficher les points sur les mains

In [ ]:
def display_hands_points(frame, multi_hand_landmarks):
    if multi_hand_landmarks:
        for hand_landmarks in multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

Fonction permettant d'afficher l'image

In [ ]:
def display_image(frame, prediction, H, W, x1, y1, x2, y2):
    if not DATASET_SHOWIMAGES:
        return
    
    ext_image = cv2.imread(DATASET_IMAGES[prediction])
    if ext_image is None:
        return
    
    box_width = x2 - x1
    box_height = y2 - y1
    
    if box_width > 0 and box_height > 0:
        resized_image = cv2.resize(ext_image, (box_width, box_height))
        
    if y1 > 0 and y2 < H and x1 > 0 and x2 < W:
        frame[y1:y2, x1:x2] = resized_image

Fonction permettant d'afficher la boîte autour de la main

In [ ]:
def display_hands_box(frame, prediction, H, W, data):
    x_ = data[::2]
    y_ = data[1::2]
    
    x1 = int(min(x_) * W - UI_RECTANGLE['padding'])
    y1 = int(min(y_) * H - UI_RECTANGLE['padding'])
    
    x2 = int(max(x_) * W + UI_RECTANGLE['padding'])
    y2 = int(max(y_) * H + UI_RECTANGLE['padding'])
    
    title = DATASET_DEFINITION[prediction]
    
    display_image(frame, prediction, H, W, x1, y1, x2, y2)
        
    text_size, _ = cv2.getTextSize(title, cv2.FONT_HERSHEY_SIMPLEX, UI_TEXT['font_scale'], UI_TEXT['font_thickness'])
    
    cv2.rectangle(frame, (x1, y1), (x2, y2), tuple(UI_RECTANGLE['color']), UI_RECTANGLE['thickness'])
    cv2.rectangle(frame, (x1 - UI_RECTANGLE['thickness'], y1 - UI_RECTANGLE['thickness'] - UI_TEXT['margin']), (x2 + UI_RECTANGLE['thickness'], y1 - text_size[1] - (UI_TEXT['padding'] * 2) - UI_TEXT['margin']), tuple(UI_RECTANGLE['color']), -1)
    
    cv2.putText(frame, title, (x1, y1 - UI_TEXT['padding'] - (UI_TEXT['margin'] * 2)), cv2.FONT_HERSHEY_SIMPLEX, UI_TEXT['font_scale'], tuple(UI_TEXT['color']), UI_TEXT['font_thickness'], cv2.LINE_AA)

Logique principal :

- Afficher les points de chaque main.
- Afficher des boites autour de chaque main avec la prédiction de notre modèle.

In [ ]:
cap = cv2.VideoCapture(0)

while True:    
    ret, frame = cap.read()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    
    display_hands_points(frame, results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks:
        
        H, W, _ = frame.shape
        for hand_landmarks in results.multi_hand_landmarks:
            data = get_hand_data(hand_landmarks.landmark)
            
            prediction = model.predict([np.asarray(data)])
            prediction_name = DATASET_DEFINITION[prediction[0]]
            
            display_hands_box(frame, prediction[0], H, W, data)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)